In [32]:
from flowobjects import Gas, Liquid, Tank, FlowComponent, FluidNode, tube_kv, solve_network_pressures, plot_pressure_ladder

e_ss = 1.5e-6

air = Gas(R= 287, gamma = 1.4, viscosity=1.8e-5)
water = Liquid(density = 1000, viscosity=0.001)
    
hp_tank = Tank(name='HP Tank', volume=3.78 * 0.001 * 2, pressure = 2e5, gas=air, gas_temp=300)
hp_tube_0 = FlowComponent(name='HP Bend 0 - SS-1610-9', kv= lambda t, P_up, P_down, rho, mu: tube_kv(P_up, P_down, rho, mu, D=0.01042, L=2, roughness=e_ss, bend_ang=0, K_extra=0))
orifice_0 = FlowComponent(name='Test Orifice', kv=500)
hp_tube_1 = FlowComponent(name='HP Bend 0 - SS-1610-9', kv= lambda t, P_up, P_down, rho, mu: tube_kv(P_up, P_down, rho, mu, D=0.01042, L=2, roughness=e_ss, bend_ang=45, K_extra=20))

hp_tank_node = FluidNode("HP Tank", tank=hp_tank, pressure=hp_tank.pressure, temperature=hp_tank.gas_temp, fluid=hp_tank.gas)
tube0_to_orifice = FluidNode("Tube 0 to Orifice", pressure=hp_tank.pressure, temperature=hp_tank.gas_temp, fluid=hp_tank.gas) 
orifice_to_tube1 = FluidNode("Orifice to Tube 1", pressure=hp_tank.pressure, temperature=hp_tank.gas_temp, fluid=hp_tank.gas) 
atmosphere_node = FluidNode("Atmosphere", pressure=101325, constant_pressure=True, temperature=300, fluid=air)

hp_tank_node.connect_nodes([(tube0_to_orifice, hp_tube_0, False)])
tube0_to_orifice.connect_nodes([(hp_tank_node, hp_tube_0, True), (orifice_to_tube1, orifice_0, False)])
orifice_to_tube1.connect_nodes([(tube0_to_orifice, orifice_0, True), (atmosphere_node, hp_tube_1, False)])
atmosphere_node.connect_nodes([(orifice_to_tube1, hp_tube_1, True)])

total_nodes = [hp_tank_node, tube0_to_orifice, orifice_to_tube1, atmosphere_node]

for node in total_nodes:
    if node.tank:
        node.pressure = node.tank.pressure
        node.temperature = node.tank.gas_temp if node.tank.gas is not None else None

        if node.tank.liquid and node.tank.mass_liquid > 0:
            node.fluid = node.tank.liquid
        else:
            node.fluid = node.tank.gas

solve_network_pressures(total_nodes, 0)

plot_pressure_ladder(total_nodes)


Node 'HP Tank': Pressure=2.00 bar, Fluid=Gas
Node 'Tube 0 to Orifice': Pressure=1.92 bar, Fluid=Gas
Node 'Orifice to Tube 1': Pressure=1.58 bar, Fluid=Gas
Node 'Atmosphere': Pressure=1.01 bar, Fluid=Gas


In [3]:

from flowobjects import tube_kv
print(tube_kv(2e5, 1e5, 900, 0.1, D=0.01, L=10, roughness=0, bend_ang=0, K_extra=0))

print(tube_kv(3e5, 1e5, 1000, 0.001, D=0.02, L=50, roughness=1e-5, bend_ang=0, K_extra=0))

print(tube_kv(200e5, 199e5, 232.3, 1.8e-5, D=0.01042, L=1, roughness=1e-6, bend_ang=0, K_extra=0))

print(tube_kv(2e5, 1e5, 1.25*2, 1.8e-5, D=0.01042, L=1, roughness=1e-6, bend_ang=0, K_extra=0))

0.09314464587774288
2.138795213557215
17.092468989087198
1440.7844456060157


In [ ]:
from flowobjects import Gas, Liquid, Tank, FlowComponent, FluidNode, tube_kv, solve_network_pressures, plot_pressure_ladder

e_ss = 1.5e-6

air = Gas(R= 287, gamma = 1.4, viscosity=1.8e-5)
water = Liquid(density = 1000, viscosity=0.001)
    
hp_tank = Tank(name='HP Tank', volume=3.78 * 0.001 * 2, pressure = 2e5, gas=air, gas_temp=300)
hp_tube_0 = FlowComponent(name='HP Bend 0 - SS-1610-9', kv= lambda t, P_up, P_down, rho, mu: tube_kv(P_up, P_down, rho, mu, D=0.01042, L=2, roughness=e_ss, bend_ang=0, K_extra=0))
orifice_0 = FlowComponent(name='Test Orifice', kv=500)
hp_tube_1 = FlowComponent(name='HP Bend 0 - SS-1610-9', kv= lambda t, P_up, P_down, rho, mu: tube_kv(P_up, P_down, rho, mu, D=0.01042, L=2, roughness=e_ss, bend_ang=45, K_extra=20))

hp_tank_node = FluidNode("HP Tank", tank=hp_tank, pressure=hp_tank.pressure, temperature=hp_tank.gas_temp, fluid=hp_tank.gas)
tube0_to_orifice = FluidNode("Tube 0 to Orifice", pressure=hp_tank.pressure, temperature=hp_tank.gas_temp, fluid=hp_tank.gas) 
orifice_to_tube1 = FluidNode("Orifice to Tube 1", pressure=hp_tank.pressure, temperature=hp_tank.gas_temp, fluid=hp_tank.gas) 
atmosphere_node = FluidNode("Atmosphere", pressure=101325, constant_pressure=True, temperature=300, fluid=air)

hp_tank_node.connect_nodes([(tube0_to_orifice, hp_tube_0, False)])
tube0_to_orifice.connect_nodes([(hp_tank_node, hp_tube_0, True), (orifice_to_tube1, orifice_0, False)])
orifice_to_tube1.connect_nodes([(tube0_to_orifice, orifice_0, True), (atmosphere_node, hp_tube_1, False)])
atmosphere_node.connect_nodes([(orifice_to_tube1, hp_tube_1, True)])

total_nodes = [hp_tank_node, tube0_to_orifice, orifice_to_tube1, atmosphere_node]

for node in total_nodes:
    if node.tank:
        node.pressure = node.tank.pressure
        node.temperature = node.tank.gas_temp if node.tank.gas is not None else None

        if node.tank.liquid and node.tank.mass_liquid > 0:
            node.fluid = node.tank.liquid
        else:
            node.fluid = node.tank.gas

solve_network_pressures(total_nodes, 0)

plot_pressure_ladder(total_nodes)
